In [ ]:
# %pip install numpy pandas librosa scikit-learn catboost matplotlib

import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:


# data_home = '/Users/dmwelch/Development/NYU/MIR/homework-4-DMW9400/testData2' # here the location you want the data to be in your Drive
# dataset_name = 'gtzan_genre'
# dataset_version = 'mini'
# dataset = mirdata.initialize(dataset_name, data_home=data_home, version=dataset_version)

def extract_features(file_path):
    y, sr = librosa.load(file_path, duration=30)
    features = np.array([])

    # MFCC
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    mfccs_mean = mfccs.mean(axis=1)
    features = np.hstack((features, mfccs_mean))

    # Chromagram
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = chroma.mean(axis=1)
    features = np.hstack((features, chroma_mean))

    # Spectral Contrast
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    contrast_mean = contrast.mean(axis=1)
    features = np.hstack((features, contrast_mean))

    # Tonnetz
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
    tonnetz_mean = tonnetz.mean(axis=1)
    features = np.hstack((features, tonnetz_mean))

    # Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(y)
    zcr_mean = zcr.mean()
    features = np.hstack((features, zcr_mean))

    # Spectral Roll-off
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    rolloff_mean = rolloff.mean()
    features = np.hstack((features, rolloff_mean))

    return features

genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
dataset = []

for genre in genres:
    genre_dir = os.path.join('genres', genre)
    for filename in os.listdir(genre_dir):
        if filename.endswith('.wav'):
            file_path = os.path.join(genre_dir, filename)
            features = extract_features(file_path)
            dataset.append([features, genre])

# Convert to DataFrame
df = pd.DataFrame(dataset, columns=['features', 'label'])

X = np.array(df['features'].tolist())
y = np.array(df['label'].tolist())

# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

ada_clf = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100,
    learning_rate=0.5,
    random_state=42
)
ada_clf.fit(X_train, y_train)

cat_clf = CatBoostClassifier(
    iterations=100,
    learning_rate=0.5,
    depth=6,
    loss_function='MultiClass',
    verbose=False,
    random_state=42
)
cat_clf.fit(X_train, y_train)

y_pred_ada = ada_clf.predict(X_test)
print("AdaBoost Classification Report:")
print(classification_report(y_test, y_pred_ada, target_names=le.classes_))

y_pred_cat = cat_clf.predict(X_test)
print("CatBoost Classification Report:")
print(classification_report(y_test, y_pred_cat, target_names=le.classes_))

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# AdaBoost Report
y_pred_ada = ada_clf.predict(X_test)
print("AdaBoost Classification Report:")
print(classification_report(y_test, y_pred_ada, target_names=le.classes_))

# CatBoost Report
y_pred_cat = cat_clf.predict(X_test)
print("CatBoost Classification Report:")
print(classification_report(y_test, y_pred_cat, target_names=le.classes_))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# AdaBoost Confusion Matrix
cm_ada = confusion_matrix(y_test, y_pred_ada)
plt.figure(figsize=(10,7))
sns.heatmap(cm_ada, annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('AdaBoost Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# CatBoost Confusion Matrix
cm_cat = confusion_matrix(y_test, y_pred_cat)
plt.figure(figsize=(10,7))
sns.heatmap(cm_cat, annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('CatBoost Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()